In [1]:
from __future__ import print_function, division
import os
import torch
import cv2
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import torch.nn.functional as F
torch.set_printoptions(linewidth=120)

import pickle as serializer


# this function seperates the different commands in 4 different dictionaries
# DataDict0 is for command0 'continue'
# DataDict1 is for command1 'left'
# DataDict2 is for command2 'forward'
# DataDict3 is for command3 'right'

# Coding of values
# command: 0 --> conintue, 1 -->left, 2 --> forward, 3 --> right. 
# direction: 0 --> left, 1 --> forward, 2 --> right.
#Steer: [0 --> -0.8] ,[1 --> -0.2],[2 --> 0],[3 --> 0.2],[4 --> 0.8].


with open('dataset.txt', 'rb') as f:
    dataset = serializer.load(f)
    
DataDictAll = {"image":[],"steer":[],"throttle":[],"direction":[],"command":[]}
DataDict0 = {"image":[],"steer":[],"throttle":[],"direction":[],"command":[]}
DataDict1 = {"image":[],"steer":[],"throttle":[],"direction":[],"command":[]}
DataDict2 = {"image":[],"steer":[],"throttle":[],"direction":[],"command":[]}
DataDict3 = {"image":[],"steer":[],"throttle":[],"direction":[],"command":[]}
[c0, c1, c2,c3, c4] = [0,0,0,0,0]
keys = dataset.keys()
for key in keys:
    for label in dataset[key]:
        #DataDict["image"].append(label[0])
        if label[3] == 'left':
            label[3] = 0
        if label[3] == 'right':
            label[3] = 2
        if label[3] == 'forward':
            label[3] = 1

        if label[1] == -0.8:
            label[1] = 0
            c0+=1
        if label[1] == -0.2:
            label[1] = 1
            c1+=1
        if label[1] == 0:
            label[1] = 2
            c2+=1
        if label[1] == 0.2:
            label[1] = 3
            c3+=1
        if label[1] == 0.8:
            label[1] = 4
            #if label[]c4 +=1
        if key == 'left':
            key = 1
            label.append(key)
            DataDict1["image"].append(label[0])
            DataDict1["steer"].append(label[1])
            DataDict1["throttle"].append(label[2])
            DataDict1["direction"].append(label[3])
            DataDict1["command"].append(label[4])
            
        if key == 'right':
            key = 3
            
            label.append(key)
            DataDict3["image"].append(label[0])
            DataDict3["steer"].append(label[1])
            DataDict3["throttle"].append(label[2])
            DataDict3["direction"].append(label[3])
            DataDict3["command"].append(label[4])
            
        if key == 'forward':
            key = 2
            
            label.append(key)
            DataDict2["image"].append(label[0])
            DataDict2["steer"].append(label[1])
            DataDict2["throttle"].append(label[2])
            DataDict2["direction"].append(label[3])
            DataDict2["command"].append(label[4])
            
        if key == 'continue':
            key = 0
            
            label.append(key)
            DataDict0["image"].append(label[0])
            DataDict0["steer"].append(label[1])
            DataDict0["throttle"].append(label[2])
            DataDict0["direction"].append(label[3])
            DataDict0["command"].append(label[4])
    
        gut = 'Bilder/'+ label[0]
        if os.path.exists(gut):
            if key == 1:
                label.append(key)
                DataDict1["image"].append(label[0])
                DataDict1["steer"].append(label[1])
                DataDict1["throttle"].append(label[2])
                DataDict1["direction"].append(label[3])
                DataDict1["command"].append(label[4])
            if key == 3:
                label.append(key)
                DataDict3["image"].append(label[0])
                DataDict3["steer"].append(label[1])
                DataDict3["throttle"].append(label[2])
                DataDict3["direction"].append(label[3])
                DataDict3["command"].append(label[4])
            if key == 2:
                label.append(key)
                DataDict2["image"].append(label[0])
                DataDict2["steer"].append(label[1])
                DataDict2["throttle"].append(label[2])
                DataDict2["direction"].append(label[3])
                DataDict2["command"].append(label[4])
            if key == 0:
                label.append(key)
                DataDict0["image"].append(label[0])
                DataDict0["steer"].append(label[1])
                DataDict0["throttle"].append(label[2])
                DataDict0["direction"].append(label[3])
                DataDict0["command"].append(label[4])
            label.append(key)
            DataDictAll["image"].append(label[0])
            DataDictAll["steer"].append(label[1])
            DataDictAll["throttle"].append(label[2])
            DataDictAll["direction"].append(label[3])
            DataDictAll["command"].append(label[4])
            
print([c0,c1,c2,c3,c4])
print(DataDict0['steer'])

with open('DataDictAll.txt', 'wb') as f:
    serializer.dump(DataDictAll, f)

with open('DataDict0.txt', 'wb') as f:
    serializer.dump(DataDict0, f)

with open('DataDict1.txt', 'wb') as f:
    serializer.dump(DataDict1, f)

with open('DataDict2.txt', 'wb') as f:
    serializer.dump(DataDict2, f)

with open('DataDict3.txt', 'wb') as f:
    serializer.dump(DataDict3, f)



In [2]:
# chose the DataSet you want to train the network on
ChosenDataset = 'DataDictHot.txt'

with open(ChosenDataset, 'rb') as f:
    dataset = serializer.load(f)

class AISProject(Dataset):
    """ AIS Project dataset."""
    def __init__(self, dictionary, root_dir, transform = None):

        self.dictionary = pd.DataFrame.from_dict(dictionary)
        self.root_dir = root_dir
        self.transform = transform
        print(len(dictionary))
        
    def __len__(self):
        return len(self.dictionary)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.dictionary.iloc[idx,0])
        image = io.imread(img_name)
        #steer = torch.tensor(self.dictionary.iloc[idx,1],dtype=torch.float32, requires_grad=True)
        steer = torch.tensor(self.dictionary.iloc[idx,1],dtype=torch.long)
        speed = torch.tensor(self.dictionary.iloc[idx,2],dtype = torch.float32)
        direction = torch.tensor(self.dictionary.iloc[idx,3],dtype = torch.long)
        command = torch.tensor(self.dictionary.iloc[idx,4],dtype = torch.uint8)
        sample = {'image':image, 'steer': steer, 'speed': speed,'direction':direction, 'command':command }
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [3]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    
    def __call__(self, sample):
        image, steer, speed, direction, command = sample['image'], sample['steer'], sample['speed'], sample['direction'], sample['command']
        image = image.transpose((2,0,1))
        return {'image': torch.from_numpy(image), 
                'steer': torch.tensor(steer),
               'speed': torch.tensor(speed),
               'direction':torch.tensor(direction),
               'command':torch.tensor(command)}

In [4]:
class RandomCrop(object):
    """Class to crop our data. (Modified RandomCrop, so that the crop isn't random anymore)"""
    def __call__(self, sample):
        image = sample['image']

        h, w = image.shape[:2]
        #new_h, new_w = output_size
        new_h = 64
        image = image[new_h:h, 0:w]
        #show_image to test whether the pictures are cropped as we wish
        #show_image(image)
        sample['image'] = image
        return sample

In [5]:
def show_image(image):
    plt.imshow(image)

'''
plt.figure()
show_image(io.imread(os.path.join('Bilder/',img_name)))
plt.show()
'''

"\nplt.figure()\nshow_image(io.imread(os.path.join('Bilder/',img_name)))\nplt.show()\n"

In [6]:
transformed_dataset = AISProject(dictionary = dataset, root_dir='Bilder/', 
                                transform = transforms.Compose([ToTensor()]))
transformed_dataset2 = AISProject(dictionary = dataset, root_dir='Bilder/', 
                                transform = transforms.Compose([ToTensor()]))
'''
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]
    
    print(i, sample)
    if i == 1:
        break
'''

5
5


'\nfor i in range(len(transformed_dataset)):\n    sample = transformed_dataset[i]\n    \n    print(i, sample)\n    if i == 1:\n        break\n'

In [7]:
TrainData = DataLoader(transformed_dataset, batch_size=32, 
                        shuffle=True, pin_memory=False,num_workers = 0)
TestData = DataLoader(transformed_dataset2, batch_size=1, 
                        shuffle=True, pin_memory=False)
'''
for batch_idx, dataset in enumerate(TrainData):
    print(batch_idx)
    print(dataset['image'].size())
    print(dataset['command'].size())
    print(dataset['direction'])
    if batch_idx == 3:
        break
'''
'''
def show_labels_batch(sample_batched):
    """Show image with labels for a batch of samples."""
    images_batch, labels_batch = \
        sample_batched['image'], sample_batched['steer']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1,2,0)))



for i_batch, sample_batched in enumerate(TrainData):
    print(i_batch, sample_batched['image'].size(), 
         sample_batched['steer'].size())

    if i_batch == 1:
        plt.figure(figsize=(40,40))
        show_labels_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break
'''
        

'\ndef show_labels_batch(sample_batched):\n    """Show image with labels for a batch of samples."""\n    images_batch, labels_batch =         sample_batched[\'image\'], sample_batched[\'steer\']\n    batch_size = len(images_batch)\n    im_size = images_batch.size(2)\n    \n    grid = utils.make_grid(images_batch)\n    plt.imshow(grid.numpy().transpose((1,2,0)))\n\n\n\nfor i_batch, sample_batched in enumerate(TrainData):\n    print(i_batch, sample_batched[\'image\'].size(), \n         sample_batched[\'steer\'].size())\n\n    if i_batch == 1:\n        plt.figure(figsize=(40,40))\n        show_labels_batch(sample_batched)\n        plt.axis(\'off\')\n        plt.ioff()\n        plt.show()\n        break\n'

In [8]:
''' Network with branches '''

device = torch.device("cuda")
# Definition of the network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)    # FYI: In the lecture I forgot to add the padding, thats why the feature size calculation was wrong
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv8_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(81920, 512)
        self.fc2 = nn.Linear(512, 512)
        #here we start the branching
        # Brach 0
        self.fc1B0 = nn.Linear(512, 256)
        self.fc2B0 = nn.Linear(256, 256)
        self.fc3B0 = nn.Linear(256, 5) # the output is direction
        
        #ToDo: implement other branches
        
        # Brach 1
        self.fc1B1 = nn.Linear(512, 256)
        self.fc2B1 = nn.Linear(256, 256)
        self.fc3B1 = nn.Linear(256, 5) # the output is direction
        # Brach 2
        self.fc1B2 = nn.Linear(512, 256)
        self.fc2B2 = nn.Linear(256, 256)
        self.fc3B2 = nn.Linear(256, 5) # the output is direction
        # Brach 3
        self.fc1B3 = nn.Linear(512, 256)
        self.fc2B3 = nn.Linear(256, 256)
        self.fc3B3 = nn.Linear(256, 5) # the output is directiontput is direction
        

    def forward(self, x, command):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)),2)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8_drop(self.conv8(x)))
        x = F.max_pool2d(x,4)
        x = x.view(-1, 81920)   # Flatten data for fully connected layer. Input size is 28*28, we have 2 pooling layers so we pool the spatial size down to 7*7. With 20 feature maps as the output of the previous conv we have in total 7x7x20 = 980 features.
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #x = F.relu(self.fc2(x))
        # choosing the branch
        out = []
        y = None
        for Idx in range(0,1):
            if command[Idx].item() == 0:
                x0 = F.relu(self.fc1B0(x))
                x0 = F.relu(self.fc2B0(x0))
                x0 = F.dropout(x0, training=self.training)
                x0 = self.fc3B0(x0)
                #print("0")
                o0 = F.log_softmax(x0, dim=1)
                return o0
            #out.append(o0.unsqueeze(0))
            elif command[Idx].item() == 1:
                x1 = F.relu(self.fc1B1(x))
                x1 = F.relu(self.fc2B1(x1))
                x1 = F.dropout(x1, training=self.training)
                x1 = self.fc3B1(x1)
                #print("1")
                o1 = F.log_softmax(x1, dim=1)
                return o1
                #out.append(o1.unsqueeze(0))
            elif command[Idx].item() == 2:
                x2 = F.relu(self.fc1B2(x))
                x2 = F.relu(self.fc2B2(x2))
                x2 = F.dropout(x2, training=self.training)
                x2 = self.fc3B2(x2)
                #print("2")
                o2 = F.log_softmax(x2, dim=1)
                return o2
                #out.append(o2.unsqueeze(0))
            elif command[Idx].item() == 3:
                x3 = F.relu(self.fc1B3(x))
                x3 = F.relu(self.fc2B3(x3))
                x3 = F.dropout(x3, training=self.training)
                x3 = self.fc3B3(x3)
                #print("3")
                o3 = F.log_softmax(x3, dim=1)
                return o3
                #out.append(o3.unsqueeze(0))
        #return out
        
        '''
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(self.conv4(x),2))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(F.max_pool2d(self.conv8_drop(self.conv8(x)), 2))
        x = x.view(-1, 81920)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)
        '''

# We create the network, shift it on the GPU and define a optimizer on its parameters
#model1 = Net().to(device)
#torch.save(model1, "Netz2.pt")
model = torch.load("Netz2.pt")
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
#optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
# This function trains the neural network for one epoch
def train(epoch):
    model.train()
    for batch_idx, sample in enumerate(TrainData):
        #print(batch_idx)
        # Move the input and target data on the GPU
        sample['image'] = sample['image'].type('torch.FloatTensor')

        data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
        command = sample['command']
        if True:
            out = []
            # Zero out gradients from previous step
            optimizer.zero_grad()
            for i in range(0,len(command)):
                output = model(data[i].unsqueeze(0), command[i])
                out.append(output)
            output = torch.cat(out)
            #output = model(data, command)
            #print(output.size())
            #print(target.size())
            #output = model(i, j)
            #for i in output:
                #print(output)
                #output = torch.cat(output)
            #print(output.size())
            #print(output.shape())
            
            loss = F.nll_loss(output, target)
            loss.backward()
            # Adjusting the parameters according to the loss function
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(TrainData.dataset),
                    100. * batch_idx / len(TrainData), loss.item()))
    torch.save(model, "Netz2.pt")

In [10]:

def test():
    #if os.path.isfile("Netz.pt"):
        #model = torch.load("Netz.pt")
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for sample in TestData:
            datacv = sample['image']
            # Move the input and target data on the GPU
            sample['image'] = sample['image'].type('torch.FloatTensor')
            
            #sample['direction'] = sample['direction'].type('torch.FloatTensor')
            data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
            command = sample['command']
            if True:
                output = model(data, command)
                test_loss += F.nll_loss(output, target).item()
                pred = output.max(1, keepdim=True)[1]
                #pred = output # get the index of the max log-probability
                '''
                if not pred.eq(target.view_as(pred)):   ## If you just want so see the failing examples
                    datacv = datacv.cpu()
                    grid = utils.make_grid(datacv)
                    grid = grid.numpy().transpose((1,2,0))
                    #plt.imshow(grid)
                    #cv_mat = data.cpu().data.squeeze().numpy()
                    #cv_mat = cv2.resize(cv_mat, (128, 640))
                    #plt.figure(figsize=(320,64))
                    cv2.imshow("test image", grid)
                    #plt.axis('off')
                    #plt.ioff()
                    #plt.show()
                    print(command.item())
                    print("Target label is : %d" % target.cpu().item())
                    print("Predicted label is : %d" % (pred.cpu().data.item()))
                    cv2.waitKey()
                 '''
                correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(TestData.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(TestData.dataset),
        100. * correct / len(TestData.dataset)))

    
def main():
    num_train_epochs = 10
    for epoch in range(1, num_train_epochs + 1):
        train(epoch)
    test()
    

if __name__=='__main__':
    main()
#torch.save(model.state_dict(), "Netz_state2.pt")

Train Epoch: 1 [0/9901 (0%)]	Loss: 1.827163
Train Epoch: 1 [320/9901 (3%)]	Loss: 1.113026
Train Epoch: 1 [640/9901 (6%)]	Loss: 0.479066
Train Epoch: 1 [960/9901 (10%)]	Loss: 0.686203
Train Epoch: 1 [1280/9901 (13%)]	Loss: 0.692143
Train Epoch: 1 [1600/9901 (16%)]	Loss: 0.393614
Train Epoch: 1 [1920/9901 (19%)]	Loss: 0.272511
Train Epoch: 1 [2240/9901 (23%)]	Loss: 0.651722
Train Epoch: 1 [2560/9901 (26%)]	Loss: 0.227900
Train Epoch: 1 [2880/9901 (29%)]	Loss: 0.705589
Train Epoch: 1 [3200/9901 (32%)]	Loss: 0.274385
Train Epoch: 1 [3520/9901 (35%)]	Loss: 0.655247
Train Epoch: 1 [3840/9901 (39%)]	Loss: 0.298184
Train Epoch: 1 [4160/9901 (42%)]	Loss: 0.305130
Train Epoch: 1 [4480/9901 (45%)]	Loss: 0.202750
Train Epoch: 1 [4800/9901 (48%)]	Loss: 0.061066
Train Epoch: 1 [5120/9901 (52%)]	Loss: 0.625941
Train Epoch: 1 [5440/9901 (55%)]	Loss: 0.126765
Train Epoch: 1 [5760/9901 (58%)]	Loss: 0.202482
Train Epoch: 1 [6080/9901 (61%)]	Loss: 0.204182
Train Epoch: 1 [6400/9901 (65%)]	Loss: 0.253248
T

Train Epoch: 6 [5120/9901 (52%)]	Loss: 0.007587
Train Epoch: 6 [5440/9901 (55%)]	Loss: 0.002273
Train Epoch: 6 [5760/9901 (58%)]	Loss: 0.003994
Train Epoch: 6 [6080/9901 (61%)]	Loss: 0.014513
Train Epoch: 6 [6400/9901 (65%)]	Loss: 0.022926
Train Epoch: 6 [6720/9901 (68%)]	Loss: 0.034561
Train Epoch: 6 [7040/9901 (71%)]	Loss: 0.126454
Train Epoch: 6 [7360/9901 (74%)]	Loss: 0.006377
Train Epoch: 6 [7680/9901 (77%)]	Loss: 0.006366
Train Epoch: 6 [8000/9901 (81%)]	Loss: 0.001688
Train Epoch: 6 [8320/9901 (84%)]	Loss: 0.052434
Train Epoch: 6 [8640/9901 (87%)]	Loss: 0.367833
Train Epoch: 6 [8960/9901 (90%)]	Loss: 0.006979
Train Epoch: 6 [9280/9901 (94%)]	Loss: 0.049295
Train Epoch: 6 [9600/9901 (97%)]	Loss: 0.046822
Train Epoch: 7 [0/9901 (0%)]	Loss: 0.023692
Train Epoch: 7 [320/9901 (3%)]	Loss: 0.009103
Train Epoch: 7 [640/9901 (6%)]	Loss: 0.009635
Train Epoch: 7 [960/9901 (10%)]	Loss: 0.002167
Train Epoch: 7 [1280/9901 (13%)]	Loss: 0.008651
Train Epoch: 7 [1600/9901 (16%)]	Loss: 0.034448
T

In [11]:
j = torch.tensor([[8],[3],[4]]).type('torch.FloatTensor')

var = [] 
for I in j:

    var.append(I.unsqueeze(0))
var_tensor = torch.cat(var)
print(var_tensor)

tensor([[8.],
        [3.],
        [4.]])


# This function trains the neural network for one epoch
def train(epoch):
    model.train()
    for batch_idx, sample in enumerate(dataloader):
        # Move the input and target data on the GPU
        sample['image'] = sample['image'].type('torch.FloatTensor')
        sample['steer'] = sample['steer'].type('torch.FloatTensor')
        
        data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
        #loss = nn.L1Loss()
        # Zero out gradients from previous step
        optimizer.zero_grad()
        output = model(data)
        #print(output.type())
        #print("target")
        #print(target.type())
        #output = output.squeeze()
        #loss = F.nll_loss(output, target)
        loss = F.smooth_l1_loss(output, target)
        #loss = loss(output, target)
        loss.backward()
        # Adjusting the parameters according to the loss function
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.item()))


device = torch.device("cpu")
# Definition of the network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)    # FYI: In the lecture I forgot to add the padding, thats why the feature size calculation was wrong
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv8_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(327680, 50)
        self.fc2 = nn.Linear(50, 5)
    
    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)),2)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8_drop(self.conv8(x)))
        x = F.max_pool2d(x,2)
        x = x.view(-1, 327680)   # Flatten data for fully connected layer. Input size is 28*28, we have 2 pooling layers so we pool the spatial size down to 7*7. With 20 feature maps as the output of the previous conv we have in total 7x7x20 = 980 features.
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        
        '''
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(self.conv4(x),2))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(F.max_pool2d(self.conv8_drop(self.conv8(x)), 2))
        x = x.view(-1, 327680)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)
        '''
# We create the network, shift it on the GPU and define a optimizer on its parameters
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
#optimizer = optim.Adam(model.parameters(), lr=0.01)

# This function trains the neural network for one epoch
def train(epoch):
    model.train()
    for batch_idx, sample in enumerate(dataloader):
        # Move the input and target data on the GPU
        sample['image'] = sample['image'].type('torch.FloatTensor')
        sample['steer'] = sample['steer'].type('torch.FloatTensor')
        
        data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
        #loss = nn.L1Loss()
        # Zero out gradients from previous step
        optimizer.zero_grad()
        output = model(data)
        #print(output.type())
        #print("target")
        #print(target.type())
        #output = output.squeeze()
        #loss = F.nll_loss(output, target)
        loss = F.smooth_l1_loss(output, target)
        #loss = loss(output, target)
        loss.backward()
        # Adjusting the parameters according to the loss function
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for sample in dataloader2:
            # Move the input and target data on the GPU
            sample['image'] = sample['image'].type('torch.FloatTensor')
            sample['steer'] = sample['steer'].type('torch.FloatTensor')
            #loss = nn.L1Loss()
            data, target = sample['image'].to(device), sample['steer'].to(device)  # .cuda() works too instead of .to(device)
            output = model(data)
            #output = output.squeeze()
            #print(output)
            #test_loss += F.nll_loss(output, target).item()
            test_loss += F.smooth_l1_loss(output, target, size_average=False).item() # sum up batch loss
            #test_loss += loss(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            #print(pred)
            print(target)
            #pred = output # get the index of the max log-probability
            '''
            if not pred.eq(target.view_as(pred)):   ## If you just want so see the failing examples
                cv_mat = data.cuda().data.squeeze().numpy()
                cv_mat = cv2.resize(cv_mat, (400, 400))
                cv2.imshow("test image", cv_mat)
                print("Target label is : %d" % target.cuda().item())
                print("Predicted label is : %d" % (pred.cuda().data.item()))
                cv2.waitKey()
            '''
            correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(dataloader2.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloader2.dataset),
        100. * correct / len(dataloader2.dataset)))

num_train_epochs = 2
for epoch in range(1, num_train_epochs + 1):
    train(epoch)

test()

### a fucntion that iter on dictionary to anlyse the dataset
ChosenDataset = 'DataDictAllOk1.txt'
with open(ChosenDataset, 'rb') as f:
    DataDictAllOk1 = serializer.load(f)
ChosenDataset = 'DataDictHot.txt'
with open(ChosenDataset, 'rb') as f:
    DataDictHot = serializer.load(f)

ctr = 0
for i in range(0, len(DataDictAllOk1['direction'])):
    if DataDictAllOk1['direction'][i] == 'left':
        DataDictAllOk1['direction'][i] = 0
    if DataDictAllOk1['direction'][i] == 'forward':
        DataDictAllOk1['direction'][i] = 1
    if DataDictAllOk1['direction'][i] == 'right':
        DataDictAllOk1['direction'][i] = 2
    ctr+=1

print(ctr)
with open('DataDictAllOk1.txt', 'wb') as f:
    serializer.dump(DataDictHot, f)           

#right on command right = 752 sharf 340
#left on command left= 600
#right on command left = 295, sharf 101
# left on command right = 101
#268 -0.8
#1845 left, 1737 right, 
# DictAllOk1 left -0.8 522, -0.2 1121, 

    

x = 'image'
y = 3
for j in range(0, len(DataDictHot[x])):
    gut = 'Bilder/'+ DataDictHot[x][j]
    if not os.path.exists(gut):
        print(bad)
print(len(DataDictHot[x]))

ChosenDataset = 'DataDictAllOk1.txt'
with open(ChosenDataset, 'rb') as f:
    DataDictAllOk1 = serializer.load(f)
ChosenDataset = 'DataDictHot.txt'
with open(ChosenDataset, 'rb') as f:
    DataDictHot = serializer.load(f)

x = 'right'
y = 3
for j in range(0, len(dataset0[x])):
    gut = 'Bilder/'+ dataset0[x][j][0]
    if os.path.exists(gut):
        DataDictHot["image"].append(dataset0[x][j][0])
        if dataset0[x][j][1] == -0.8:
            DataDictHot["steer"].append(0)
        elif dataset0[x][j][1] == -0.2:
            DataDictHot["steer"].append(1)
        elif dataset0[x][j][1] == 0:
            DataDictHot["steer"].append(2)
        elif dataset0[x][j][1] == 0.2:
            DataDictHot["steer"].append(3)
        elif dataset0[x][j][1] == 0.8:
            DataDictHot["steer"].append(4)
        DataDictHot["throttle"].append(dataset0[x][j][2])
        DataDictHot["direction"].append(dataset0[x][j][3])
        DataDictHot["command"].append(y)
        #print(dataset0['right'][j])
        ctr2 +=1
     

print(len(DataDictHot['command']))
print(len(DataDictAllOk1['command']))